# IMPORT

In [20]:
import json
import asyncio
from tqdm import tqdm
from playwright.async_api import async_playwright
from bs4 import BeautifulSoup

# FUNCTION

In [ ]:
class PlaywrightScrolling():
    def __init__(self):
        self.playwright = None
        self.browser = None
        self.page = None
        self.url = None
    
    async def initialize(self, url, resume_page=None):
        """Initialize the playwright instance and browser"""
        self.url = url
        self.playwright = await async_playwright().start()
        self.browser = await self.playwright.chromium.launch(headless=False)
        if resume_page:
            self.page = resume_page
        else:
            context = await self.browser.new_context(
                user_agent='Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
            )
            self.page = await context.new_page()
            
            # Set longer timeout and wait for network idle
            try:
                print(f"Đang truy cập: {url}")
                await self.page.goto(url, wait_until='domcontentloaded', timeout=500000)
                print("Truy cập thành công!")
            except Exception as e:
                print(f"Lỗi khi truy cập trang: {e}")
                raise
    
    
    async def scroll(self, time_sleep=2, num_trials=3):
        async def press_end():
            await self.page.keyboard.press("End")
            await asyncio.sleep(time_sleep)
        
        prev_height = await self.page.evaluate("document.body.scrollHeight")
        await press_end()
        # Check if height changed
        trial = 0
        while True:
            if trial == num_trials:
                break
            new_height = await self.page.evaluate("document.body.scrollHeight")
            if new_height == prev_height:
                trial += 1
                print(f"Cannot scroll - trial {trial}")
                await press_end()
            else:
                prev_height = new_height
                break
        
        
    async def get_page_content(self):
        return await self.page.content()
    
    async def close(self):
        """Close browser and playwright"""
        if self.browser:
            await self.browser.close()
        if self.playwright:
            await self.playwright.stop()

# CRAWL

In [22]:
scroller = PlaywrightScrolling()
url = "https://vn.investing.com/news/stock-market-news/2001-doc-gi-truoc-gio-giao-dich-chung-khoan-2514598"
await scroller.initialize(url=url)

NotImplementedError: 